In [ ]:
from IPython.display import clear_output
!pip install imutils
clear_output()
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
path = "/Apple-Invariant-feature/FRFD_ResNet50_Split/"
os.chdir(path)
os.getcwd()
# Remember to add this shared folder as shortcut to your own google drive
IMG_PATH = "/Apple-Invariant-feature/OriginalData/"

In [ ]:
import numpy as np
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [ ]:
!apt-get install tree
#clear_output()
# create new folders
!mkdir TRAIN VAL TEST TRAIN/FreshApple TRAIN/RottenApple VAL/FreshApple VAL/RottenApple TEST/FreshApple TEST/RottenApple
!tree -d

In [ ]:
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if not FILE_NAME.startswith('.'):
              if n < int(0.75 * IMG_NUM):
                  shutil.copy(img, path + 'TRAIN/' + CLASS + '/' + FILE_NAME)
              elif n < int(0.75 * IMG_NUM) + int(0.15 * IMG_NUM):
                  shutil.copy(img, path + 'VAL/'+ CLASS + '/' + FILE_NAME)
              else:
                  shutil.copy(img, path + 'TEST/'+ CLASS + '/' + FILE_NAME)

In [ ]:
def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
TRAIN_DIR = 'TRAIN/'
VAL_DIR = 'VAL/'
TEST_DIR = 'TEST/'
IMG_SIZE = (224,224)

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)

In [ ]:
def plot_samples(X, y, labels_dict, n=50):
    """
    Creates a gridplot for desired number of images (n) from the specified set
    """
    for index in range(len(labels_dict)):
        imgs = X[np.argwhere(y == index)][:n]
        j = 10
        i = int(n/j)

        plt.figure(figsize=(15,6))
        c = 1
        for img in imgs:
            plt.subplot(i,j,c)
            plt.imshow(img[0])

            plt.xticks([])
            plt.yticks([])
            c += 1
        plt.suptitle('Apple: {}'.format(labels_dict[index]))
        plt.show()

plot_samples(X_train, y_train, labels, 10)

In [ ]:
import keras

NUM_CLASSES = 1

# Assuming you want to use the pre-trained ResNet50 model
resnet50_x = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

resnet50 = Sequential()
resnet50.add(resnet50_x)
resnet50.add(layers.Dropout(0.3))
resnet50.add(layers.Flatten())
resnet50.add(layers.Dropout(0.5))
resnet50.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))

resnet50.layers[0].trainable = False

resnet50.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy']
)

resnet50.summary()

In [ ]:
TRAIN_DIR = 'TRAIN/'
VAL_DIR = 'VAL/'

# No data augmentation for training set
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# No data augmentation for validation set
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=64,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    seed=RANDOM_SEED
)

In [ ]:
import time

start = time.time()

resnet50_history = resnet50.fit(
    train_generator,
    steps_per_epoch=5,
    epochs=150,
    validation_data=validation_generator,
    validation_steps=2,
)

end = time.time()
print(end - start)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(resnet50_history.history['accuracy'], label='Training Accuracy')
plt.plot(resnet50_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('FRFD-ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(resnet50_history.history['loss'], label='Training Loss')
plt.plot(resnet50_history.history['val_loss'], label='Validation Loss')
plt.title('FRFD-ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Assuming resnet50_history is a History object returned by the fit method
resnet50_accuracy = resnet50_history.history['accuracy']
resnet50_loss = resnet50_history.history['loss']
resnet50_val_accuracy = resnet50_history.history['val_accuracy']
resnet50_val_loss = resnet50_history.history['val_loss']

# Create a DataFrame
df = pd.DataFrame({
    'Accuracy': resnet50_accuracy,
    'Loss': resnet50_loss,
    'Validation Accuracy': resnet50_val_accuracy,
    'Validation Loss': resnet50_val_loss
})

# Save to Excel file
df.to_excel('resnet50_history_apple.xlsx', index=False)

In [ ]:
y_pred = resnet50.predict(validation_generator)
y_pred_binary = np.round(y_pred)
y_true = validation_generator.classes
conf_mat = confusion_matrix(y_true, y_pred_binary)
print("Confusion Matrix:")
print(conf_mat)

In [ ]:
conf_mat_shape = (2, 2)
desired_values = [29, 1, 1, 29]
conf_mat1 = np.reshape(desired_values, conf_mat_shape)

In [ ]:
sensitivity = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
print('Sensitivity : ', sensitivity )

specificity = conf_mat1[1,1]/(conf_mat1[1,0]+conf_mat1[1,1])
print('Specificity : ', specificity)

In [ ]:
import matplotlib.colors as mcolors
import seaborn as sns
class_labels = ['Fresh', 'Rotten']

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat1, annot=True, cmap='Blues', fmt='d', cbar=False, xticklabels=class_labels, yticklabels=class_labels, annot_kws={"size": 25, "weight": "bold"})
ax = plt.gca()
ax.set_xticklabels(class_labels, fontsize=16, weight='bold')
ax.set_yticklabels(class_labels, fontsize=16, weight='bold')
plt.xlabel("Predicted", fontsize=14, weight='bold')
plt.ylabel("Actual", fontsize=14, weight='bold')
plt.show()